In [1]:
import argparse
import numpy as np
import pandas as pd

from pathlib import Path

from goscripts import obo_tools
from goscripts import gaf_parser

from phppipy.dataprep import taxonid
from phppipy.ppi_tools import id_mapper
from phppipy.ppi_tools import label_interpro
from phppipy.ppi_tools import label_go
from phppipy.ppi_tools import ppi_filter

In [2]:
obo_path = Path('/media/pieter/DATA/Wetenschap/Doctoraat/projects/host-pathogen-ppi-analysis/data/raw/go_data/go.obo')
go_dict = obo_tools.importOBO(obo_path, ignore_part_of=False)

Retrieved 47179 GO terms from /media/pieter/DATA/Wetenschap/Doctoraat/projects/host-pathogen-ppi-analysis/data/raw/go_data/go.obo 

Adding secondary GO identifiers...



In [3]:
obo_tools.buildGOtree(go_dict, root_nodes=['GO:0008150', 'GO:0005575', 'GO:0003674'])

In [4]:
ppi_file = Path('/media/pieter/DATA/Wetenschap/Doctoraat/projects/host-pathogen-ppi-analysis/data/interim/10292/ppi_data/ppi-filter-remap.tsv')
ppi_df = pd.read_csv(ppi_file, sep='\t', header=0)
print('PPIs were imported from {}\n'.format(ppi_file))
protein_set = set(ppi_df.xref_A.append(ppi_df.xref_B, ignore_index=True).str.extract('^.*:(\w*)-?', expand=False).unique())

gaf_path = Path('/media/pieter/DATA/Wetenschap/Doctoraat/projects/host-pathogen-ppi-analysis/data/interim/10292/go_data/goa_uniprot.gaf')
gaf_dict = gaf_parser.importGAF(gaf_path, protein_set)

PPIs were imported from /media/pieter/DATA/Wetenschap/Doctoraat/projects/host-pathogen-ppi-analysis/data/interim/10292/ppi_data/ppi-filter-remap.tsv

Retrieved 2995 annotated (background filtered) Uniprot AC's from /media/pieter/DATA/Wetenschap/Doctoraat/projects/host-pathogen-ppi-analysis/data/interim/10292/go_data/goa_uniprot.gaf



In [5]:
gaf_path = Path('/media/pieter/DATA/Wetenschap/Doctoraat/projects/host-pathogen-ppi-analysis/data/interim/all/go_data/goa_uniprot.gaf')
gaf_dict2 = gaf_parser.importGAF(gaf_path, protein_set)

Retrieved 2995 annotated (background filtered) Uniprot AC's from /media/pieter/DATA/Wetenschap/Doctoraat/projects/host-pathogen-ppi-analysis/data/interim/all/go_data/goa_uniprot.gaf



In [6]:
{key for key in gaf_dict if key not in gaf_dict2}

set()

In [7]:
interpro_path = Path('/media/pieter/DATA/Wetenschap/Doctoraat/projects/host-pathogen-ppi-analysis/data/interim/all/interpro/protein2ipr.dat')
interpro_dict = label_interpro.create_uniprot2interpro_dict(interpro_path, protein_set)
label_interpro.annotate_interpro(ppi_df, interpro_dict)
ppi_df.interpro_xref_A

0            {IPR000198, IPR036865, IPR008936, IPR001251}
1                                  {IPR029199, IPR026667}
2                       {IPR011992, IPR018247, IPR002048}
3       {IPR034769, IPR035979, IPR034772, IPR012677, I...
4       {IPR028889, IPR016024, IPR001394, IPR038765, I...
5                                             {IPR004882}
6            {IPR013026, IPR019734, IPR011990, IPR007192}
7            {IPR016024, IPR003890, IPR009818, IPR016021}
8            {IPR006594, IPR006595, IPR013144, IPR024964}
9                       {IPR000086, IPR015797, IPR016706}
10                                            {IPR007854}
11                                 {IPR029199, IPR026668}
12      {IPR006652, IPR008979, IPR006595, IPR015915, I...
13      {IPR011989, IPR016024, IPR025640, IPR035445, I...
14      {IPR036028, IPR036871, IPR035558, IPR037426, I...
15                                            {IPR031887}
16      {IPR036028, IPR036871, IPR035557, IPR001452, I...
17           {

In [8]:
ppi_df.xref_A

0       uniprotkb:Q07960
1       uniprotkb:Q9Y2W1
2       uniprotkb:Q06190
3       uniprotkb:Q16630
4       uniprotkb:O00507
5       uniprotkb:O95232
6       uniprotkb:Q9UJX2
7       uniprotkb:Q9H074
8       uniprotkb:Q9NWU2
9       uniprotkb:O43809
10      uniprotkb:Q6UN15
11      uniprotkb:Q9NYF8
12      uniprotkb:Q9UL63
13      uniprotkb:O75165
14      uniprotkb:Q9Y5X1
15      uniprotkb:Q86SQ7
16      uniprotkb:Q96RF0
17      uniprotkb:Q96PY6
18      uniprotkb:O43464
19      uniprotkb:Q9P2I0
20      uniprotkb:Q8N5Z5
21      uniprotkb:Q7L5Y9
22      uniprotkb:Q93008
23      uniprotkb:Q9C0J8
24      uniprotkb:Q14681
25      uniprotkb:Q9NQH7
26      uniprotkb:P62875
27      uniprotkb:P52434
28      uniprotkb:O94763
29      uniprotkb:Q9H6T3
              ...       
7721    uniprotkb:A8K4C8
7722    uniprotkb:A8K5I0
7723    uniprotkb:B2R4S9
7724    uniprotkb:A4D1U3
7725    uniprotkb:E9KL48
7726    uniprotkb:Q13546
7727    uniprotkb:P06400
7728    uniprotkb:Q96ST3
7729    uniprotkb:Q9H4A3


In [9]:
protein_set = set(ppi_df.xref_A.append(ppi_df.xref_B, ignore_index=True).str.extract('^.*:(\w*)-?', expand=False).unique())

In [10]:
protein_set2 = set(pd.Series(pd.unique(ppi_df[['xref_A','xref_B']].values.ravel('K'))).str.extract('^.*:(\w*)-?', expand=False))

In [11]:
protein_set == protein_set2

True

In [12]:
label_go.annotate_GO(ppi_df, gaf_dict)
ppi_df2 = ppi_df.copy()
label_go.annotate_GO(ppi_df2, gaf_dict2)

In [13]:
all(ppi_df[['xref_A_GO', 'xref_B_GO']] == ppi_df2[['xref_A_GO', 'xref_B_GO']])

KeyError: "['xref_A_GO' 'xref_B_GO'] not in index"

In [ ]:
ppi_df['xref_A'].str.extract('^.*:(\w*)-?', expand=False).apply(lambda x: gaf_dict.get(x, np.NaN))
ppi_df['xref_A'].str.extract('^.*:(\w*)-?', expand=False).map(gaf_dict)

In [ ]:
import argparse
import numpy as np
import pandas as pd
import sys

from pathlib import Path

from phppipy.ppi_tools import id_mapper
from phppipy.ppi_tools import ppi_import
from phppipy.ppi_tools import ppi_filter
from phppipy.dataprep import taxonid
input_dir = Path('/media/pieter/DATA/Wetenschap/Doctoraat/projects/host-pathogen-ppi-analysis/data/raw/ppi_data')
mitab_files = input_dir.glob('*.mitab')
ppi_df_list = [ppi_import.read_mi_tab(i) for i in mitab_files if i.is_file()]
phisto_files = input_dir.glob('phi*.csv')
mi_file = input_dir / 'mi.obo'
ppi_df_list.extend(
    [ppi_import.read_mitab_phisto(i, mi_file) for i in phisto_files])
for i in ppi_df_list:
    i['origin'] = i.name
ppi_df_all = pd.concat(ppi_df_list, axis=0, join='outer', ignore_index=True)
# Merge PPI datasets
for i in ppi_df_list:
    i['origin'] = i.name
ppi_df_all = pd.concat(ppi_df_list, axis=0, join='outer', ignore_index=True)

In [14]:
id_mapper.check_unique_identifier(ppi_df_all)
id_mapper.map2uniprot(ppi_df_all, Path('/media/pieter/DATA/Wetenschap/Doctoraat/projects/host-pathogen-ppi-analysis/data/interim/all/mapping'), reviewed_only=True, skip_creation=True)

NameError: name 'ppi_df_all' is not defined

In [ ]:
'uniprotkb:F5H9W9' in ppi_df_all.xref_B.values

In [15]:
protein_set = set(ppi_df.xref_A.append(ppi_df.xref_B, ignore_index=True).str.extract('^.*:(\w*)-?', expand=False).unique())

In [16]:
protein_set2 = set(pd.Series(pd.unique(ppi_df[['xref_A','xref_B']].values.ravel('K'))).str.extract('^.*:(\w*)-?', expand=False))

In [17]:
protein_set == protein_set2

True

In [18]:
label_go.annotate_GO(df_herpes, gaf_dict)


NameError: name 'df_herpes' is not defined

In [23]:
d = ppi_df.loc[1, ['interpro_xref_A','interpro_xref_B']]
d

interpro_xref_A               {IPR029199, IPR026667}
interpro_xref_B    {IPR027417, IPR001889, IPR013672}
Name: 1, dtype: object

In [21]:
ppi_df.loc[ppi_df['interpro_xref_B'].isnull(), 'interpro_xref_B'] = ppi_df.loc[ppi_df['interpro_xref_B'].isnull(), 'interpro_xref_B'].apply(lambda x: set())
ppi_df.loc[ppi_df['interpro_xref_A'].isnull(), 'interpro_xref_A'] = ppi_df.loc[ppi_df['interpro_xref_A'].isnull(), 'interpro_xref_A'].apply(lambda x: set())
ppi_df.loc[ppi_df['GO_xref_B'].isnull(), 'GO_xref_B'] = ppi_df.loc[ppi_df['GO_xref_B'].isnull(), 'GO_xref_B'].apply(lambda x: set())
ppi_df.loc[ppi_df['GO_xref_A'].isnull(), 'GO_xref_A'] = ppi_df.loc[ppi_df['GO_xref_A'].isnull(), 'GO_xref_A'].apply(lambda x: set())


In [24]:
set().union(*d)

{'IPR001889', 'IPR013672', 'IPR026667', 'IPR027417', 'IPR029199'}

In [ ]:
ppi_df[['interpro_xref_A','interpro_xref_B']].apply(lambda x: set().union(*x), axis=1)

In [ ]:
import itertools
type(list(itertools.combinations('ABC', 2))[0])

In [ ]:
set([tuple(sorted(i)) for i in itertools.product([1,2,3],[2,3])])

In [ ]:
list(((x,y) for x in [1,2,3] for y in [2,3]))

In [ ]:
list((x,y) for x in set([1,2,3]) for y in set([2,3]))

In [ ]:
set().union(*ppi_df[['GO_xref_A',]])

In [ ]:
annotation_set = set().union(*ppi_df['GO_xref_B']) | set().union(*ppi_df['GO_xref_A'])

In [ ]:
'IPR001251' in annotation_set

In [ ]:
annotation_set2 = set().union(*ppi_df[['GO_xref_A', 'GO_xref_B']].values.ravel('K'))

In [ ]:
annotation_set == annotation_set2

In [ ]:
len(list(itertools.combinations(annotation_set, 2)))

In [ ]:
set([tuple(sorted(i)) for i in itertools.product(ppi_df['GO_xref_A'],ppi_df['GO_xref_B'])])

In [ ]:
ppi_df['annotation_pairs'] = ppi_df[['GO_xref_A', 'GO_xref_B']].apply(lambda x:
                                         set([tuple(sorted(i)) for i in itertools.product(x['GO_xref_A'],x['GO_xref_B'])]), axis=1)

In [ ]:
len(set([j for i in pairs for j in i]))

In [ ]:
len(set().union(*pairs))

In [ ]:
type()

In [ ]:
np.sum(ppi_df['annotation_pairs'].apply(lambda x: ('GO:0016197', 'GO:0055036') in x))

In [ ]:
 pairs = set().union(*ppi_df['annotation_pairs'])

In [ ]:
list(pairs)[2][1]

In [ ]:
pairs_set.map(('GO:0016020', 'GO:0048471') in pairs_set)

In [ ]:
pd.Series(list(range(10))).map(str)

In [ ]:
ppi_df[['GO_xref_A', 'GO_xref_B']].apply(lambda x: set().union(*x), axis=1)

In [25]:
def merge_annotations(interaction_dataframe, columns):
    """Merges the annotations for a protein-protein interaction.

    Given two columns of a protein-protein interaction dataframe, each of
    which contains a type of annotation data, this function returns the merged
    set of those annotations.

    E.g. Column 1: {IPR011333, IPR003131, IPR000210}
         Column 2: {GO:0046872}
         Result: {GO:0046872, IPR011333, IPR003131, IPR000210}

    Parameters
    ----------
    interaction_dataframe : DataFrame
        DataFrame containing protein-protein interactions and annotations.
    columns : list
        A list of annotation columns to merge. Expects the column contents to
        be array-like, not strings.

    Returns
    -------
    pandas Series of sets
        Array containing a set of annotations for each interaction (row).
    """

    # replace NaNs with empty sets
    for i in columns:
        interaction_dataframe.loc[interaction_dataframe[
            i].isnull(), i] = interaction_dataframe.loc[interaction_dataframe[
                i].isnull(), i].apply(lambda x: set())

    # join all sets in each supplied column
    # the unpacking operator can accept an array of lists or sets
    merged_annotations = interaction_dataframe[columns].apply(
        lambda x: set().union(*x), axis=1)

    return merged_annotations

In [27]:
merged_annotations_A

0       {GO:0010008, GO:0070062, IPR036865, GO:0048471...
1       {GO:0006351, GO:0070062, GO:0007623, GO:004851...
2       {GO:0019888, GO:0006470, GO:0001754, GO:009024...
3       {GO:0016607, GO:0035061, GO:1990904, IPR035979...
4       {GO:0016787, GO:0016579, GO:0070410, GO:000728...
5       {GO:0016607, GO:0005685, GO:0003729, GO:000838...
6       {GO:0007049, GO:0000278, IPR011990, GO:0004842...
7       {GO:0045727, GO:0048255, GO:0005737, GO:000641...
8       {IPR013144, IPR024964, IPR006595, GO:0005634, ...
9       {GO:0005815, GO:0005737, GO:1990120, GO:004280...
10      {GO:0000398, GO:0006406, GO:0005847, GO:000372...
11      {GO:0006351, GO:0043620, GO:0016607, GO:000573...
12      {GO:0007160, GO:0042995, IPR006652, IPR008979,...
13      {GO:0010008, GO:0043231, GO:0001649, GO:007006...
14      {GO:0032461, GO:0031625, GO:0030659, GO:007006...
15      {GO:0000086, GO:0035148, GO:0005737, GO:009771...
16      {GO:0010008, GO:0007049, GO:0000278, GO:003065...
17      {GO:00

In [26]:
merged_annotations_A = merge_annotations(ppi_df, ['GO_xref_A', 'interpro_xref_A'])
labeled_merged_annotations_A = merged_annotations_A.map(lambda x: set(['h@' + i for i in x]))
ppi_df['labeled_merged_annotations_A'] = labeled_merged_annotations_A
merged_annotations_B = merge_annotations(ppi_df, ['GO_xref_B', 'interpro_xref_B'])
labeled_merged_annotations_B = merged_annotations_B.map(lambda x: set(['p@' + i for i in x]))
ppi_df['labeled_merged_annotations_B'] = labeled_merged_annotations_B
labeled_merged_annotations_B

0       {p@GO:0019012, p@GO:0019031, p@GO:0055036, p@G...
1       {p@IPR013672, p@IPR001889, p@IPR027417, p@GO:0...
2       {p@IPR013672, p@IPR001889, p@IPR027417, p@GO:0...
3       {p@IPR013672, p@IPR001889, p@IPR027417, p@GO:0...
4       {p@IPR013672, p@IPR001889, p@IPR027417, p@GO:0...
5       {p@IPR013672, p@IPR001889, p@IPR027417, p@GO:0...
6       {p@IPR013672, p@IPR001889, p@IPR027417, p@GO:0...
7       {p@IPR013672, p@IPR001889, p@IPR027417, p@GO:0...
8       {p@IPR013672, p@IPR001889, p@IPR027417, p@GO:0...
9       {p@IPR013672, p@IPR001889, p@IPR027417, p@GO:0...
10      {p@IPR013672, p@IPR001889, p@IPR027417, p@GO:0...
11      {p@IPR013672, p@IPR001889, p@IPR027417, p@GO:0...
12      {p@IPR013672, p@IPR001889, p@IPR027417, p@GO:0...
13      {p@IPR013672, p@IPR001889, p@IPR027417, p@GO:0...
14      {p@IPR013672, p@IPR001889, p@IPR027417, p@GO:0...
15      {p@IPR013672, p@IPR001889, p@IPR027417, p@GO:0...
16      {p@IPR013672, p@IPR001889, p@IPR027417, p@GO:0...
17      {p@IPR

In [ ]:
test_df = pd.concat([labeled_merged_annotations_A, labeled_merged_annotations_B], axis=1)
test_df

In [ ]:
test_dummies = test_df[0].apply(lambda x: pd.Series([1] * len(x), index=x)).fillna(0, downcast='infer')

In [ ]:
test_dummies = pd.concat([i.apply(lambda x: pd.Series([1] * len(x), index=x)).fillna(0, downcast='infer') for i in [labeled_merged_annotations_A]], axis=1)

In [ ]:
test_dummies

In [ ]:
test_dummies2 = pd.concat([i.apply(lambda x: pd.Series([1] * len(x), index=x)).fillna(0, downcast='infer') for i in [labeled_merged_annotations_B]], axis=1)

In [ ]:
dummies_df = pd.concat([test_dummies, test_dummies2], axis=1)

In [36]:
import itertools
columns = ['labeled_merged_annotations_A', 'labeled_merged_annotations_B']

In [37]:
ppi_df[columns].apply(lambda x: set([tuple(sorted(i)) for i in itertools.product(x[columns[0]], x[columns[1]])]), axis=1)

0       {(h@GO:0017137, p@GO:0019012), (h@GO:0005515, ...
1       {(h@GO:0008380, p@GO:0071897), (h@GO:0051219, ...
2       {(h@GO:0007525, p@GO:0071897), (h@GO:0061053, ...
3       {(h@GO:0005737, p@GO:0016740), (h@GO:0098789, ...
4       {(h@GO:0036459, p@GO:0009157), (h@GO:0005737, ...
5       {(h@GO:0008380, p@GO:0071897), (h@GO:0003729, ...
6       {(h@IPR019734, p@GO:0006230), (h@GO:0007080, p...
7       {(h@GO:0005737, p@GO:0016740), (h@IPR016024, p...
8       {(h@IPR006595, p@GO:0016740), (h@GO:0030054, p...
9       {(h@GO:0005737, p@GO:0016740), (h@GO:0098789, ...
10      {(h@GO:0098789, p@GO:0009157), (h@GO:0006406, ...
11      {(h@GO:0043065, p@GO:0016301), (h@IPR026668, p...
12      {(h@IPR010565, p@GO:0016301), (h@GO:0005737, p...
13      {(h@GO:0016020, p@GO:0004797), (h@IPR001623, p...
14      {(h@GO:0005737, p@GO:0016740), (h@GO:0000278, ...
15      {(h@GO:0005737, p@GO:0016740), (h@GO:0007098, ...
16      {(h@IPR035703, p@GO:0004797), (h@GO:0000278, p...
17      {(h@GO

In [ ]:
len(pairs)

In [ ]:
import time

In [ ]:
a = time.time()

In [ ]:
b = time.time()
b - a

In [ ]:
import itertools
ppi_df['annotation_pairs'] = ppi_df[['labeled_merged_annotations_A', 'labeled_merged_annotations_B']].apply(lambda x:
                                         set([tuple(sorted(i)) for i in itertools.product(x['labeled_merged_annotations_A'],x['labeled_merged_annotations_B'])]), axis=1)

In [ ]:
pairs = set().union(*ppi_df['annotation_pairs'])

In [ ]:
ppi_df.annotation_pairs

In [ ]:
len(pairs)

In [ ]:
annotation_set = set().union(*ppi_df['labeled_merged_annotations_A', 'labeled_merged_annotations_B'].values.ravel('K'))
list(itertools.combinations(annotation_set, 2))

In [ ]:
ppi_df.shape[0]

In [ ]:
len(pairs)

In [29]:
ppi_df['labeled_merged_annotations_B']

0       {p@GO:0019012, p@GO:0019031, p@GO:0055036, p@G...
1       {p@IPR013672, p@IPR001889, p@IPR027417, p@GO:0...
2       {p@IPR013672, p@IPR001889, p@IPR027417, p@GO:0...
3       {p@IPR013672, p@IPR001889, p@IPR027417, p@GO:0...
4       {p@IPR013672, p@IPR001889, p@IPR027417, p@GO:0...
5       {p@IPR013672, p@IPR001889, p@IPR027417, p@GO:0...
6       {p@IPR013672, p@IPR001889, p@IPR027417, p@GO:0...
7       {p@IPR013672, p@IPR001889, p@IPR027417, p@GO:0...
8       {p@IPR013672, p@IPR001889, p@IPR027417, p@GO:0...
9       {p@IPR013672, p@IPR001889, p@IPR027417, p@GO:0...
10      {p@IPR013672, p@IPR001889, p@IPR027417, p@GO:0...
11      {p@IPR013672, p@IPR001889, p@IPR027417, p@GO:0...
12      {p@IPR013672, p@IPR001889, p@IPR027417, p@GO:0...
13      {p@IPR013672, p@IPR001889, p@IPR027417, p@GO:0...
14      {p@IPR013672, p@IPR001889, p@IPR027417, p@GO:0...
15      {p@IPR013672, p@IPR001889, p@IPR027417, p@GO:0...
16      {p@IPR013672, p@IPR001889, p@IPR027417, p@GO:0...
17      {p@IPR

In [28]:
merged_annotations = merge_annotations(ppi_df, ['labeled_merged_annotations_A', 'labeled_merged_annotations_B'])
merged_annotations

0       {h@GO:0017137, h@GO:2001136, p@GO:0019012, h@G...
1       {h@GO:0038023, h@GO:0005654, h@GO:0000381, p@I...
2       {h@GO:0006470, p@IPR013672, h@GO:0090263, h@GO...
3       {h@GO:0005849, h@GO:0005654, p@IPR013672, h@GO...
4       {h@GO:0016787, p@IPR013672, h@GO:0070410, h@GO...
5       {h@GO:0005654, p@IPR013672, h@GO:0071004, h@GO...
6       {h@GO:0005654, h@GO:0007080, p@IPR013672, h@GO...
7       {p@IPR013672, p@GO:0004797, p@GO:0006230, h@IP...
8       {h@GO:0005654, p@IPR013672, p@GO:0004797, p@GO...
9       {h@GO:0005849, h@GO:0005654, p@IPR013672, h@GO...
10      {h@GO:0005654, p@IPR013672, h@GO:0005847, h@GO...
11      {h@GO:0005654, p@IPR013672, h@GO:0043065, h@GO...
12      {p@IPR013672, h@GO:0001726, h@IPR010565, p@GO:...
13      {h@GO:0007032, p@IPR013672, h@GO:0015031, h@IP...
14      {h@GO:0045860, p@IPR013672, h@GO:0071933, h@GO...
15      {p@IPR013672, h@GO:0005814, h@GO:0005856, p@GO...
16      {p@IPR013672, h@GO:0015031, p@GO:0004797, h@GO...
17      {h@GO:

In [ ]:
a = np.array(['a','b','c'])
b = np.array(['a','d'])
set(a).isdisjoint(b)
# np.intersect1d(a,b) # must be len(0)
# np.setdiff1d(b,a) # must be len(2)

In [ ]:
not('a' in a and 'd' in a)

In [ ]:
pair[0]

In [ ]:
merged_annotations.map(lambda x: 'h@GO:0009967' not in x and 'h@GO:0048511' not in x)

In [ ]:
pair = list(pairs)[0]
print(pair)
presence_mask = ppi_df['annotation_pairs'].apply(lambda x: pair in x)
pair_count = np.sum(presence_mask)
pair_count2 = np.sum(merged_annotations.map(lambda x: pair[0] in x and pair[1] in x))
label_one_count = np.sum(merged_annotations.apply(lambda x: pair[0] in x and pair[1] not in x))
label_two_count = np.sum(merged_annotations.apply(lambda x: pair[0] not in x and pair[1] in x))
absent_count = np.sum(ppi_df['annotation_pairs'].apply(lambda x: pair not in x))
absent_count2 = np.sum(merged_annotations.map(lambda x: pair[0] not in x and pair[1] not in x))

In [ ]:
print(pair_count)
print(pair_count2)
print(label_one_count)
print(label_two_count)
print(absent_count)
print(absent_count2)
print(ppi_df.shape[0])
pair_count + label_one_count + label_two_count + absent_count2

In [ ]:
dummies_df['p@IPR015164']

In [ ]:
np.histogram2d(dummies_df['h@GO:0045656'], dummies_df['p@IPR015164'], 100)[0]

In [ ]:
xedges = [0, 1, 3, 5]
yedges = [0, 2, 3, 4, 6]
x = np.random.normal(2, 1, 100)
y = np.random.normal(1, 1, 100)
H, xedges, yedges = np.histogram2d(x, y, bins=(xedges, yedges))

In [ ]:
H

In [ ]:
from scipy.stats import chi2_contingency
obs = np.array([[10, 10, 20], [20, 20, 20]])
obs = np.array([[57,87,24], [50, 42, 6,], [42, 22, 5]])
obs

In [ ]:
chi2, p, dof, ex = chi2_contingency(obs, correction=False)
chi2

In [ ]:
p

In [ ]:
{pair : {'jaccard': {}, 'pmi': {}, 'G': {}, 'chi2': {}} for pair in pairs}[('h@GO:0045656', 'p@IPR015164')]['jaccard']

In [ ]:
import math
math.log((1159/50000952) / ((1938/50000952)*(1311/50000952)))



In [ ]:
d = { 1:  {'jaccard': 'sdgsd', 'pmi': [1,2], 'G': {}}, 2:  {'jaccard': {}, 'pmi': {}, 'G': {}}}

In [ ]:
pd.DataFrame.from_dict(d, orient='index')

In [ ]:
results_dict = {
    # 'jaccard': {},
    'pmi': {},
    'G': {},
    'chi2': {},
    'fisher': {},
    'phi': {},
    'min_count': {}
}

In [ ]:
import scipy
pair = list(pairs)[0]
def _calc_pmi(n11, n1plus, nplus1):
    return math.log(n11 / (n1plus * nplus1), 2)


def _calc_chi2(n11, n10, n01, n00):
    contingency_table = np.array([[n11, n01], [n10, n00]])
    chi2, p, df, ex = scipy.stats.chi2_contingency(
        contingency_table, correction=False, lambda_="pearson")
    d = {'chi2': chi2, 'p-value': p, 'df': df, 'exp': ex}
    return d


def _calc_G(n11, n10, n01, n00):
    contingency_table = np.array([[n11, n01], [n10, n00]])
    G, p, df, ex = scipy.stats.chi2_contingency(
        contingency_table, correction=False, lambda_="log-likelihood")
    d = {'G': G, 'p-value': p, 'df': df, 'exp': ex}
    return d


def _calc_fisher(n11, n10, n01, n00):
    contingency_table = np.array([[n11, n01], [n10, n00]])
    oddsratio, p = scipy.stats.fisher_exact(contingency_table)
    d = {'oddsration': oddsratio, 'p-value': p}
    return d

interaction_dataframe = ppi_df
presence_mask = interaction_dataframe['annotation_pairs'].apply(
        lambda x: pair in x)
pair_count = np.sum(presence_mask)

# count ppis where only 1 label occurs: P(X | Y') or N10
label_one_count_exclusive = np.sum(
    merged_annotations.apply(
        lambda x: pair[0] in x and pair[1] not in x))
label_two_count_exclusive = np.sum(
    merged_annotations.apply(
        lambda x: pair[0] not in x and pair[1] in x))

# count ppis where 1 label occurs, regardless of other label in pair:
# P(X) or N1+
label_one_count = np.sum(
    merged_annotations.apply(lambda x: pair[0] in x))
label_two_count = np.sum(
    merged_annotations.apply(lambda x: pair[1] in x))

# TODO: should b,c label counts count PPIs with 1 occurrence or pairs with 1 occurrence? the former probably
# label_one_count = np.sum(pd.Series(list(pairs)).apply(lambda x: pair[0] in x))
# label_two_count = np.sum(pd.Series(list(pairs)).apply(lambda x: pair[1] in x))

# count ppis lacking either term: P(X',Y') or N00
absent_count = np.sum(
    merged_annotations.map(
        lambda x: pair[0] not in x and pair[1] not in x))
total_count = interaction_dataframe.shape[0]

# results_dict[pair]['pmi'] = _calc_pmi(pair_count, label_one_count, label_two_count)
# results_dict[pair]['chi2'] = _calc_chi2(pair_count, label_one_count_exclusive, label_two_count_exclusive, absent_count)
# results_dict[pair]['G'] = _calc_G(pair_count, label_one_count_exclusive, label_two_count_exclusive, absent_count)
# results_dict[pair]['fisher'] = _calc_fisher(pair_count, label_one_count_exclusive, label_two_count_exclusive, absent_count)

results_dict['pmi'][pair] = _calc_pmi(pair_count, label_one_count,
                                      label_two_count)
results_dict['chi2'][pair] = _calc_chi2(
    pair_count, label_one_count_exclusive, label_two_count_exclusive,
    absent_count)
results_dict['G'][pair] = _calc_G(
    pair_count, label_one_count_exclusive, label_two_count_exclusive,
    absent_count)
results_dict['fisher'][pair] = _calc_fisher(
    pair_count, label_one_count_exclusive, label_two_count_exclusive,
    absent_count)

results_dict['min_count'][pair] = min([
    pair_count, label_one_count_exclusive, label_two_count_exclusive,
    absent_count
])


In [ ]:
pd.DataFrame.from_dict(results_dict['chi2'], orient='index')

In [ ]:
for name, d in results_dict.items():
    print(name, pd.DataFrame.from_dict(d, orient='index').add_prefix(name + '_'))

In [ ]:
df_list = [pd.DataFrame.from_dict(d, orient='index').add_prefix(name + '_') for name, d in results_dict.items()]
results = pd.concat(df_list, axis=1)

In [ ]:
results

In [ ]:
import statsmodels.stats.multitest
p_values = results['G_p-value'].values
corrected_p = statsmodels.stats.multitest.multipletests(p_values, alpha=0.05, method='fdr_bh')
print(p_values)
print(corrected_p)

In [ ]:
results['G_p-value' + '_' + 'fdr'] = corrected_p[1]

In [ ]:
results

In [ ]:
d = {'col1': [1, 2], 'col2': [3, 4]}
df = pd.DataFrame(d)
df['col3'] = df['col1'].map(lambda x: set([1,22]))

In [ ]:
df

In [ ]:
df.to_csv('/home/pieter/Documents/df-test.tsv', sep='\t', index=False, header=True)

In [ ]:
import ast
df_read = pd.read_csv('/home/pieter/Documents/df-test.tsv', sep='\t', header=0, converters={"col3": ast.literal_eval})

In [ ]:
type(df_read.loc[0,'col3'])

In [ ]:
import ast
df_read = pd.read_csv('/home/pieter/Documents/df-test.tsv', sep='\t', header=0)
df_read.col3[0]

In [ ]:
df_read.col3.map(ast.literal_eval)

In [ ]:
sci

In [ ]:
new = pd.read_csv('/media/pieter/DATA/Wetenschap/Doctoraat/projects/host-pathogen-ppi-analysis/data/interim/10292/ppi_data/ppi-annotations.tsv', 
                  sep='\t', header=0)

In [ ]:
new.head()

In [ ]:
test = ppi_df.GO_xref_A.apply(lambda x: pd.Series([1] * len(x), index=x)).fillna(0, downcast='infer')

In [ ]:
test[test['GO:0000012'] == 1]

In [ ]:
ppi_df.GO_xref_A[1143]

In [ ]:
test_df = ppi_df.loc[0:5,['GO_xref_A']].copy()
# test_df.loc[1, 'GO_xref_A'] = np.NaN
test_df.loc[2, 'GO_xref_A'] = set()
test_df

In [ ]:
test_df.GO_xref_A.apply(lambda x: pd.Series([1] * len(x), index=x)).fillna(0, downcast='infer')

In [ ]:
test_df.loc[1].values[0]

In [ ]:
ppi_df.merge

# Speed comparison of annotation presence counters

In [1]:
from math import sqrt
from joblib import Parallel, delayed

import importlib
import itertools

import scipy

import argparse
import numpy as np
import pandas as pd
import ast
import math

from pathlib import Path

from goscripts import obo_tools
from goscripts import gaf_parser

from phppipy.dataprep import taxonid
from phppipy.ppi_tools import id_mapper
from phppipy.ppi_tools import label_interpro
from phppipy.ppi_tools import label_go
from phppipy.ppi_tools import ppi_filter
import argparse
import numpy as np
import pandas as pd
import sys

from pathlib import Path

from phppipy.ppi_tools import id_mapper
from phppipy.ppi_tools import ppi_import
from phppipy.ppi_tools import ppi_filter
from phppipy.dataprep import taxonid

sys.path.append('/media/pieter/DATA/Wetenschap/Doctoraat/projects/host-pathogen-ppi-analysis/src')
from phppipy.mining import pairwise
importlib.reload(pairwise)

<module 'phppipy.mining.pairwise' from '/media/pieter/DATA/Wetenschap/Doctoraat/projects/host-pathogen-ppi-analysis/src/phppipy/mining/pairwise.py'>

Label annotations and merge them into 'annotations_A' and 'annotations_B' columns. 

In [2]:
ppi_file = Path('/media/pieter/DATA/Wetenschap/Doctoraat/projects/host-pathogen-ppi-analysis/data/interim/10292/ppi_data/ppi-annotations.tsv')
ppi_df = pd.read_csv(
    ppi_file,
    sep='\t',
    header=0)
print('PPIs were imported from {}\n'.format(ppi_file))

for i in ['GO_xref_A', 'GO_xref_B', 'interpro_xref_A', 'interpro_xref_B']:
    ppi_df.loc[~ppi_df[i].isnull(), i] = ppi_df.loc[~ppi_df[i].isnull(), i].map(ast.literal_eval)

merged_annotations_A = pairwise.merge_annotations(
    ppi_df, ['GO_xref_A', 'interpro_xref_A'])
labeled_merged_annotations_A = pairwise.add_hp_label(merged_annotations_A, 'h')
ppi_df['annotations_A'] = labeled_merged_annotations_A

merged_annotations_B = pairwise.merge_annotations(
    ppi_df, ['GO_xref_B', 'interpro_xref_B'])
labeled_merged_annotations_B = pairwise.add_hp_label(merged_annotations_B, 'p')
ppi_df['annotations_B'] = labeled_merged_annotations_B

PPIs were imported from /media/pieter/DATA/Wetenschap/Doctoraat/projects/host-pathogen-ppi-analysis/data/interim/10292/ppi_data/ppi-annotations.tsv



Create pairs of annotations and also a merged set across both proteins

In [3]:
interaction_dataframe = ppi_df   # incase name is different due to copying from function definitions
columns = ['annotations_A', 'annotations_B']

# replace NaNs by empty sets
for i in columns:
    interaction_dataframe.loc[interaction_dataframe[
        i].isnull(), i] = interaction_dataframe.loc[interaction_dataframe[
            i].isnull(), i].apply(lambda x: set())
    
interaction_dataframe['annotation_pairs'] = interaction_dataframe[columns].apply(lambda x: set([tuple(sorted(i)) for i in itertools.product(x[columns[0]], x[columns[1]])]), axis=1)
# next, these lists are joined and duplicates are removed
pairs = set().union(*interaction_dataframe['annotation_pairs'])

# create a set of annotations for each interaction as a whole (i.e. join
# labels for the two interacting proteins)
merged_annotations = pairwise.merge_annotations(interaction_dataframe, columns)

results_dict = {
    # 'jaccard': {},
    'pmi': {},
    'G': {},
    'chi2': {},
    'fisher': {},
    'phi': {},
    'min_count': {}
}

In [4]:
# Create Gene Ontology dictionaries
print('Creating Gene Ontology dictionaries...')
obo_path = Path('/media/pieter/DATA/Wetenschap/Doctoraat/projects/host-pathogen-ppi-analysis/data/raw/go_data/go.obo')
go_dict = obo_tools.importOBO(obo_path, ignore_part_of=False)

# TODO: allow selection of GO namespaces
obo_tools.buildGOtree(
    go_dict, root_nodes=['GO:0008150', 'GO:0005575', 'GO:0003674'])

Creating Gene Ontology dictionaries...
Retrieved 47179 GO terms from /media/pieter/DATA/Wetenschap/Doctoraat/projects/host-pathogen-ppi-analysis/data/raw/go_data/go.obo 

Adding secondary GO identifiers...



Define pairwise measure functions

In [5]:
def _calc_pmi(n11, n1plus, nplus1, nplusplus):
    return math.log((n11/nplusplus) /
                    ((n1plus/nplusplus) * (nplus1/nplusplus)),
                    2)


def _calc_chi2(n11, n10, n01, n00):
    contingency_table = np.array([[n11, n01], [n10, n00]])
    chi2, p, df, ex = scipy.stats.chi2_contingency(
        contingency_table, correction=False, lambda_="pearson")
    d = {'chi2': chi2, 'p-value': p, 'df': df, 'exp': ex}
    return d


def _calc_G(n11, n10, n01, n00):
    contingency_table = np.array([[n11, n01], [n10, n00]])
    G, p, df, ex = scipy.stats.chi2_contingency(
        contingency_table, correction=False, lambda_="log-likelihood")
    d = {'G': G, 'p-value': p, 'df': df, 'exp': ex}
    return d


def _calc_fisher(n11, n10, n01, n00):
    contingency_table = np.array([[n11, n01], [n10, n00]])
    oddsratio, p = scipy.stats.fisher_exact(contingency_table)
    d = {'oddsration': oddsratio, 'p-value': p}
    return d

## Singleton pair approach

## Pair is present

In [13]:
%time np.sum(interaction_dataframe['annotation_pairs'].apply(lambda x: pair in x))
%time np.sum(interaction_dataframe.apply(lambda x: pair[0] in x['annotations_A'] and pair[1] in x['annotations_B'], axis=1))
%time np.sum(interaction_dataframe.apply(lambda x: pair in list(itertools.product(x['annotations_A'], x['annotations_B'])), axis=1))

CPU times: user 3.61 ms, sys: 0 ns, total: 3.61 ms
Wall time: 2.98 ms
CPU times: user 113 ms, sys: 0 ns, total: 113 ms
Wall time: 113 ms
CPU times: user 586 ms, sys: 0 ns, total: 586 ms
Wall time: 584 ms


6

## Exactly one label is present

In [19]:
pair = list(pairs)[0]
%time  np.sum(merged_annotations.apply(lambda x: pair[0] in x and pair[1] not in x))
%time  np.sum(merged_annotations.map(lambda x: pair[0] in x and pair[1] not in x))
%time  np.sum(merged_annotations.map(lambda x: pair[0] in x) & merged_annotations.map(lambda x: pair[1] not in x))
ppi_df['annotations_A_str'] = ppi_df.annotations_A.map(str)
ppi_df['annotations_B_str'] = ppi_df.annotations_B.map(str)
%time  np.sum(ppi_df.loc[(ppi_df.annotations_A_str.str.contains(pair[0])) & ~(ppi_df.annotations_A_str.str.contains(pair[0]))])
%time  np.sum(ppi_df.apply(lambda x: pair[0] in x['annotations_A'] and pair[1] not in x['annotations_B'], axis=1))

CPU times: user 3.01 ms, sys: 90 µs, total: 3.1 ms
Wall time: 2.77 ms
CPU times: user 3.14 ms, sys: 8 µs, total: 3.15 ms
Wall time: 2.89 ms
CPU times: user 6.82 ms, sys: 0 ns, total: 6.82 ms
Wall time: 6.19 ms
CPU times: user 14.1 ms, sys: 0 ns, total: 14.1 ms
Wall time: 13.8 ms
CPU times: user 101 ms, sys: 0 ns, total: 101 ms
Wall time: 100 ms


29

## At least one label is present

In [22]:
pair = list(pairs)[0]
%time np.sum(merged_annotations.apply(lambda x: pair[0] in x))
%time np.sum(merged_annotations.map(lambda x: pair[0] in x))
%time np.sum(ppi_df.loc[(ppi_df.annotations_A_str.str.contains(pair[0]))])

CPU times: user 3.76 ms, sys: 0 ns, total: 3.76 ms
Wall time: 3.58 ms
CPU times: user 3.81 ms, sys: 0 ns, total: 3.81 ms
Wall time: 3.3 ms
CPU times: user 21.5 ms, sys: 39 µs, total: 21.5 ms
Wall time: 20.5 ms


Series([], dtype: float64)

## Neither label

In [68]:
%time np.sum(merged_annotations.map(lambda x: pair[0] not in x and pair[1] not in x))
%time np.sum(merged_annotations.map(lambda x: pair[0] not in x) & merged_annotations.map(lambda x: pair[1] not in x))

CPU times: user 7.6 ms, sys: 0 ns, total: 7.6 ms
Wall time: 7.42 ms
CPU times: user 8.44 ms, sys: 11 µs, total: 8.45 ms
Wall time: 7.73 ms


4973

## Parallelization

In [90]:
def count_stuff(pair, interaction_dataframe, pairs_column, merged_annotations):
    
    # count all ppis where pair occurs
    presence_mask = interaction_dataframe[pairs_column].apply(
        lambda x: pair in x)
    pair_count = np.sum(presence_mask)

    # count ppis where only 1 label occurs: P(X | Y') or N10
    label_one_count_exclusive = np.sum(
        merged_annotations.apply(
            lambda x: pair[0] in x and pair[1] not in x))
    label_two_count_exclusive = np.sum(
        merged_annotations.apply(
            lambda x: pair[0] not in x and pair[1] in x))

    # count ppis where 1 label occurs, regardless of other label in pair:
    # P(X) or N1+
    label_one_count = np.sum(
        merged_annotations.apply(lambda x: pair[0] in x))
    label_two_count = np.sum(
        merged_annotations.apply(lambda x: pair[1] in x))

    # count ppis lacking either term: P(X',Y') or N00
    absent_count = np.sum(
        merged_annotations.map(
            lambda x: pair[0] not in x and pair[1] not in x))
    total_count = interaction_dataframe.shape[0]
    
#     pmi = _calc_pmi(pair_count, label_one_count, label_two_count, total_count)
#     chi2 = _calc_chi2(pair_count, label_one_count_exclusive, label_two_count_exclusive, absent_count)
#     G = _calc_G(pair_count, label_one_count_exclusive, label_two_count_exclusive, absent_count)
#     fisher = _calc_fisher(pair_count, label_one_count_exclusive, label_two_count_exclusive, absent_count)
    
#     return (pmi, chi2, G, fisher)

In [72]:
short_pairs = list(pairs)[0:100]
list_pairs = list(pairs)
from math import sqrt
from joblib import Parallel, delayed
%time Parallel(n_jobs=7)(delayed(count_stuff)(i, ppi_df, columns, merged_annotations) for i in short_pairs)
%time Parallel(n_jobs=7,backend="threading")(delayed(count_stuff)(i, ppi_df, columns, merged_annotations) for i in short_pairs)
%time [count_stuff(i, ppi_df, columns, merged_annotations) for i in short_pairs]

CPU times: user 1min 17s, sys: 6.13 s, total: 1min 23s
Wall time: 1min 46s
CPU times: user 1.73 s, sys: 61.4 ms, total: 1.79 s
Wall time: 1.73 s
CPU times: user 1.33 s, sys: 0 ns, total: 1.33 s
Wall time: 1.33 s


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [73]:
short_pairs = list(pairs)[0:1000]
list_pairs = list(pairs)
from math import sqrt
from joblib import Parallel, delayed
%time Parallel(n_jobs=7)(delayed(count_stuff)(i, ppi_df, columns, merged_annotations) for i in short_pairs)
%time Parallel(n_jobs=7,backend="threading")(delayed(count_stuff)(i, ppi_df, columns, merged_annotations) for i in short_pairs)
%time [count_stuff(i, ppi_df, columns, merged_annotations) for i in short_pairs]

CPU times: user 13min 15s, sys: 1min 3s, total: 14min 19s
Wall time: 18min 4s
CPU times: user 17.9 s, sys: 441 ms, total: 18.4 s
Wall time: 17 s
CPU times: user 13.6 s, sys: 5 ms, total: 13.6 s
Wall time: 13.6 s


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [ ]:
# DONT DO THIS - MEMORY ERROR!
# short_pairs = list(pairs)[0:1000]
# list_pairs = list(pairs)
# # df_copies = [ppi_df.copy() for i in range(len(pairs))]
# from math import sqrt
# from joblib import Parallel, delayed
# %time Parallel(n_jobs=7,backend="threading")(delayed(count_stuff)(i, ppi_df, columns, merged_annotations) for i, j in zip(short_pairs, df_copies))
# %time [count_stuff(i, ppi_df, columns, merged_annotations) for i, j in zip(short_pairs, df_copies)]

# Set-based presence checking

In [23]:
propagated_pairs = []
for pair in list(pairs)[:100]:
    new_pair = []
    for term in pair:
        if 'GO' in term:
            prefix = term[:2]
            go_object = go_dict.get(term[2:])
            if None:
                new_pair.append(term)
            else:
                child_terms = [prefix + i for i in go_object.recursive_children]
                propagation_set = set(child_terms) | set([term])
                new_pair.append(propagation_set)
        else:
            new_pair.append({term})
    propagated_pairs.append(tuple(new_pair))

pair_prop = propagated_pairs[0]
pair_prop

({'h@GO:0008157',
  'h@GO:0019902',
  'h@GO:0019903',
  'h@GO:0030346',
  'h@GO:0051721',
  'h@GO:1990634'},
 {'p@GO:0019034',
  'p@GO:0039715',
  'p@GO:0039720',
  'p@GO:0039721',
  'p@GO:0042025',
  'p@GO:0044094',
  'p@GO:0044095',
  'p@GO:0044196',
  'p@GO:0044199',
  'p@GO:0044200',
  'p@GO:0044201',
  'p@GO:0044202',
  'p@GO:0044203',
  'p@GO:0044204',
  'p@GO:0046809',
  'p@GO:0072495',
  'p@GO:0075341'})

## Complete pair is present

In [25]:
%time np.sum([np.sum(interaction_dataframe['annotation_pairs'].apply(lambda x: i in x)) for i in list(itertools.product(pair_prop[0], pair_prop[1]))])
%time np.sum(interaction_dataframe.apply(lambda x: not x['annotations_A'].isdisjoint(pair_prop[0]) and not x['annotations_B'].isdisjoint(pair_prop[1]), axis=1))
%time np.sum(merged_annotations.map(lambda x: not x.isdisjoint(pair_prop[0]) and not x.isdisjoint(pair_prop[1])))
%time np.sum(merged_annotations.apply(lambda x: not x.isdisjoint(pair_prop[0]) and not x.isdisjoint(pair_prop[1])))

CPU times: user 254 ms, sys: 0 ns, total: 254 ms
Wall time: 252 ms
CPU times: user 108 ms, sys: 0 ns, total: 108 ms
Wall time: 108 ms
CPU times: user 4.64 ms, sys: 0 ns, total: 4.64 ms
Wall time: 4.5 ms
CPU times: user 4.09 ms, sys: 0 ns, total: 4.09 ms
Wall time: 3.96 ms


46

## Exactly one label is present

No difference between Series.map(), list comprehension of .values array converted to list, list comprehension of values array converted to list and re-numpy-arrayed, or generator expression of .values array.

In [26]:
a = ppi_df.annotations_A.values
%time np.sum([i.isdisjoint(pair_prop[0]) for i in a])
%time np.sum(np.array([i.isdisjoint(pair_prop[0]) for i in a]))
%time np.sum(np.fromiter((i.isdisjoint(pair_prop[0]) for i in a), dtype=bool))


CPU times: user 3.94 ms, sys: 0 ns, total: 3.94 ms
Wall time: 3.82 ms
CPU times: user 4.04 ms, sys: 0 ns, total: 4.04 ms
Wall time: 3.92 ms
CPU times: user 4.31 ms, sys: 0 ns, total: 4.31 ms
Wall time: 4.22 ms


7546

In [27]:
pair_prop = propagated_pairs[0]
%time np.sum(ppi_df.apply(lambda x: not x['annotations_A'].isdisjoint(pair_prop[0]) and x['annotations_B'].isdisjoint(pair_prop[1]), axis=1))
%time np.sum(ppi_df.annotations_A.map(lambda x: not x.isdisjoint(pair_prop[0])) & ppi_df.annotations_B.map(lambda x: x.isdisjoint(pair_prop[1])))

# https://jakevdp.github.io/PythonDataScienceHandbook/02.06-boolean-arrays-and-masks.html

CPU times: user 131 ms, sys: 13 µs, total: 131 ms
Wall time: 130 ms
CPU times: user 7.05 ms, sys: 0 ns, total: 7.05 ms
Wall time: 6.68 ms


159

In [65]:
# compare methods' output
all(ppi_df.annotations_A.map(lambda x: not x.isdisjoint(pair_prop[0])) & ppi_df.annotations_B.map(lambda x: x.isdisjoint(pair_prop[1])) == ppi_df.apply(lambda x: not x['annotations_A'].isdisjoint(pair_prop[0]) and x['annotations_B'].isdisjoint(pair_prop[1]), axis=1))

True

## Atleast one label

In [59]:
%time np.sum(merged_annotations.apply(lambda x: not x.isdisjoint(pair_prop[0])))
%time np.sum(merged_annotations.map(lambda x: not x.isdisjoint(pair_prop[0])))
%time np.sum(interaction_dataframe[columns[0]].map(lambda x: not x.isdisjoint(pair_prop[0])))

CPU times: user 12.4 ms, sys: 9 µs, total: 12.4 ms
Wall time: 10.9 ms
CPU times: user 8.36 ms, sys: 9 µs, total: 8.37 ms
Wall time: 7.91 ms
CPU times: user 5.38 ms, sys: 0 ns, total: 5.38 ms
Wall time: 5.05 ms


205

In [45]:
merged_annotations.index[merged_annotations.apply(lambda x: not x.isdisjoint(pair_prop[0]))]

Int64Index([  41,   44,   68,   89,  141,  162,  245,  251,  271,  474,
            ...
            7567, 7574, 7576, 7619, 7621, 7627, 7643, 7705, 7706, 7729],
           dtype='int64', length=205)

In [47]:
merged_annotations[41]

{'h@GO:0000164',
 'h@GO:0000775',
 'h@GO:0000776',
 'h@GO:0000777',
 'h@GO:0000784',
 'h@GO:0003723',
 'h@GO:0004721',
 'h@GO:0004722',
 'h@GO:0005515',
 'h@GO:0005521',
 'h@GO:0005634',
 'h@GO:0005654',
 'h@GO:0005694',
 'h@GO:0005730',
 'h@GO:0005737',
 'h@GO:0005739',
 'h@GO:0005741',
 'h@GO:0005829',
 'h@GO:0005925',
 'h@GO:0005975',
 'h@GO:0005977',
 'h@GO:0006470',
 'h@GO:0007049',
 'h@GO:0008022',
 'h@GO:0008157',
 'h@GO:0016607',
 'h@GO:0016787',
 'h@GO:0016791',
 'h@GO:0019901',
 'h@GO:0019903',
 'h@GO:0019904',
 'h@GO:0030182',
 'h@GO:0030496',
 'h@GO:0032154',
 'h@GO:0032922',
 'h@GO:0032991',
 'h@GO:0042752',
 'h@GO:0043153',
 'h@GO:0043197',
 'h@GO:0044877',
 'h@GO:0046822',
 'h@GO:0046872',
 'h@GO:0047485',
 'h@GO:0048511',
 'h@GO:0051301',
 'h@GO:0060252',
 'h@GO:0072357',
 'h@IPR004843',
 'h@IPR006186',
 'h@IPR029052',
 'h@IPR031675',
 'h@IPR037981',
 'p@GO:0005515',
 'p@IPR006772'}

In [48]:
merged_annotations[41] & pair_prop[0]

{'h@GO:0008157', 'h@GO:0019903'}

Check if methods give same output

In [50]:
np.sum(interaction_dataframe[columns[0]].map(lambda x: not x.isdisjoint(pair_prop[0])))

205

In [51]:
np.sum(merged_annotations.apply(lambda x: not x.isdisjoint(pair_prop[0])))

205

## Neither label is present

In [85]:
%time np.sum(merged_annotations.map(lambda x: pair[0] not in x and pair[1] not in x))
%time np.sum(merged_annotations.map(lambda x: x.isdisjoint(pair_prop[0]) and x.isdisjoint(pair_prop[1])))
%time np.sum(merged_annotations.map(lambda x: x.isdisjoint(pair_prop[0])) & merged_annotations.map(lambda x: x.isdisjoint(pair_prop[1])))
%time np.sum(interaction_dataframe[columns[0]].map(lambda x: x.isdisjoint(pair_prop[0])) & interaction_dataframe[columns[1]].map(lambda x: x.isdisjoint(pair_prop[1])))

CPU times: user 8.6 ms, sys: 0 ns, total: 8.6 ms
Wall time: 7.76 ms
CPU times: user 9.75 ms, sys: 3.79 ms, total: 13.5 ms
Wall time: 12.8 ms
CPU times: user 11.1 ms, sys: 8 µs, total: 11.1 ms
Wall time: 10.7 ms
CPU times: user 7.3 ms, sys: 0 ns, total: 7.3 ms
Wall time: 7.13 ms


4746

Compare methods output

In [87]:
all(merged_annotations.map(lambda x: x.isdisjoint(pair_prop[0])) & merged_annotations.map(lambda x: x.isdisjoint(pair_prop[1])) == interaction_dataframe[columns[0]].map(lambda x: x.isdisjoint(pair_prop[0])) & interaction_dataframe[columns[1]].map(lambda x: x.isdisjoint(pair_prop[1])))

True

## Map is generally (slightly) faster than apply, even for a Series!

In [54]:
%time np.sum(merged_annotations.apply(lambda x: pair[0] in x))
%time np.sum(merged_annotations.map(lambda x: pair[0] in x))

CPU times: user 259 µs, sys: 3.98 ms, total: 4.24 ms
Wall time: 3.62 ms
CPU times: user 4.05 ms, sys: 0 ns, total: 4.05 ms
Wall time: 3.87 ms


85

## Parallellization

In [93]:
def count_presences_propagated(pair_prop, interaction_dataframe, column_A,
                               column_B, merged_annotations):
    """
    columns must be host, pathogen order
    """
    """Comparison of methods

    ALL PAIRS
    %time np.sum([np.sum(interaction_dataframe['annotation_pairs'].apply(lambda x: i in x)) for i in list(itertools.product(pair_prop[0], pair_prop[1]))])
    %time np.sum(interaction_dataframe.apply(lambda x: not x['annotations_A'].isdisjoint(pair_prop[0]) and not x['annotations_B'].isdisjoint(pair_prop[1]), axis=1))
    %time np.sum(merged_annotations.map(lambda x: not x.isdisjoint(pair_prop[0]) and not x.isdisjoint(pair_prop[1])))

    CPU times: user 138 ms, sys: 30 µs, total: 138 ms
    Wall time: 136 ms
    CPU times: user 105 ms, sys: 0 ns, total: 105 ms
    Wall time: 105 ms
    CPU times: user 8.65 ms, sys: 0 ns, total: 8.65 ms
    Wall time: 8.46 ms

    ONLY 1 LABEL

    """

    # count all ppis where any pair occurs
    pair_count = np.sum(
        merged_annotations.map(
            lambda x: not x.isdisjoint(pair_prop[0]) and not x.isdisjoint(pair_prop[1])
        ))

    # count ppis where only 1 label occurs: P(X | Y') or N10
    label_one_count_exclusive = np.sum(interaction_dataframe[column_A].map(
        lambda x: not x.isdisjoint(pair_prop[0])
    ) & interaction_dataframe[column_B].map(
        lambda x: x.isdisjoint(pair_prop[1])))
    label_two_count_exclusive = np.sum(interaction_dataframe[column_A].map(
        lambda x: x.isdisjoint(pair_prop[0])
    ) & interaction_dataframe[column_B].map(
        lambda x: not x.isdisjoint(pair_prop[1])))

    # count ppis where 1 label occurs, regardless of other label in pair:
    # P(X) or N1+
    label_one_count = np.sum(
        merged_annotations.apply(lambda x: not x.isdisjoint(pair_prop[0])))
    label_two_count = np.sum(
        merged_annotations.apply(lambda x: not x.isdisjoint(pair_prop[1])))
    # option to not discern between location of label (i.e. host or pathogen)
    # in case there would be identical labels in host and pathogen proteins
    # label_one_count = np.sum(interaction_dataframe[column_A].map(
    #     lambda x: not x.isdisjoint(pair_prop[0])))
    # label_two_count = np.sum(interaction_dataframe[column_B].map(
    #     lambda x: not x.isdisjoint(pair_prop[1])))

    # count ppis lacking either term: P(X',Y') or N00
    absent_count = np.sum(
        merged_annotations.map(lambda x: x.isdisjoint(pair_prop[0])) &
        merged_annotations.map(lambda x: x.isdisjoint(pair_prop[1])))
    # option to not discern between location of label (i.e. host or pathogen)
    # absent_count = np.sum(interaction_dataframe[column_A].map(
    #     lambda x: x.isdisjoint(pair_prop[0])) & interaction_dataframe[column_B]
    #                       .map(lambda x: x.isdisjoint(pair_prop[1])))

    # total count of terms
    total_count = interaction_dataframe.shape[0]

    return pair_count, label_one_count_exclusive, label_two_count_exclusive, label_one_count, label_two_count, absent_count, total_count


In [96]:
from joblib import Parallel, delayed
short_pairs_prop = list(propagated_pairs)[0:100]
columns = ['annotations_A', 'annotations_B']
%time Parallel(n_jobs=7)(delayed(count_presences_propagated)(i, ppi_df, 'annotations_A', 'annotations_B', merged_annotations) for i in short_pairs_prop)
%time Parallel(n_jobs=7,backend="threading")(delayed(count_presences_propagated)(i, ppi_df, 'annotations_A', 'annotations_B', merged_annotations) for i in short_pairs_prop)
%time [count_presences_propagated(i, ppi_df, 'annotations_A', 'annotations_B', merged_annotations) for i in short_pairs_prop]

CPU times: user 1min 12s, sys: 8.45 s, total: 1min 20s
Wall time: 1min 24s
CPU times: user 4.37 s, sys: 59 ms, total: 4.43 s
Wall time: 4.31 s
CPU times: user 3.37 s, sys: 0 ns, total: 3.37 s
Wall time: 3.37 s


[(46, 159, 2800, 205, 2846, 4746, 7751),
 (2, 2, 155, 4, 157, 7592, 7751),
 (1, 69, 63, 70, 64, 7618, 7751),
 (3, 32, 5, 35, 8, 7711, 7751),
 (6, 32, 329, 38, 335, 7384, 7751),
 (1, 9, 99, 10, 100, 7642, 7751),
 (2, 31, 295, 33, 297, 7423, 7751),
 (1, 3, 182, 4, 183, 7565, 7751),
 (1, 5, 159, 6, 160, 7586, 7751),
 (2, 0, 24, 2, 26, 7725, 7751),
 (23, 679, 84, 702, 107, 6965, 7751),
 (25, 114, 1385, 139, 1410, 6227, 7751),
 (3, 167, 222, 170, 225, 7359, 7751),
 (97, 658, 429, 755, 526, 6567, 7751),
 (1, 0, 322, 1, 323, 7428, 7751),
 (4, 173, 43, 177, 47, 7531, 7751),
 (1, 2, 297, 3, 298, 7451, 7751),
 (13, 278, 1094, 291, 1107, 6366, 7751),
 (3, 75, 376, 78, 379, 7297, 7751),
 (1, 13, 147, 14, 148, 7590, 7751),
 (1, 5, 19, 6, 20, 7726, 7751),
 (3, 46, 376, 49, 379, 7326, 7751),
 (3, 351, 84, 354, 87, 7313, 7751),
 (2, 0, 550, 2, 552, 7199, 7751),
 (2, 9, 77, 11, 79, 7663, 7751),
 (5, 87, 298, 92, 303, 7361, 7751),
 (2, 8, 897, 10, 899, 6844, 7751),
 (2, 18, 285, 20, 287, 7446, 7751),
 (

'h@GO:0005654'

In [109]:
pair_prop = ({'h@GO:0042382', 'h@GO:0071182', 'h@GO:1990483', 'h@GO:0032221', 'h@GO:0016605', 'h@GO:0061793', 'h@GO:0044545', 'h@GO:0008024', 'h@GO:0015030', 'h@GO:0034967', 'h@GO:0035267', 'h@GO:0090571', 'h@GO:0000438', 'h@GO:0048188', 'h@GO:0070176', 'h@GO:0046695', 'h@GO:0005847', 'h@GO:0000123', 'h@GO:0071178', 'h@GO:0070775', 'h@GO:1990331', 'h@GO:0070449', 'h@GO:0061773', 'h@GO:0071180', 'h@GO:0005673', 'h@GO:0070824', 'h@GO:0016593', 'h@GO:0016604', 'h@GO:1902562', 'h@GO:0070692', 'h@GO:0005666', 'h@GO:0036410', 'h@GO:0035061', 'h@GO:1990512', 'h@GO:0070823', 'h@GO:0034740', 'h@GO:0070776', 'h@GO:0016580', 'h@GO:0000420', 'h@GO:0000508', 'h@GO:0044666', 'h@GO:0070440', 'h@GO:0033255', 'h@GO:0070443', 'h@GO:0090573', 'h@GO:0032044', 'h@GO:0072589', 'h@GO:0070847', 'h@GO:0000443', 'h@GO:0071179', 'h@GO:1990826', 'h@GO:0070439', 'h@GO:0071175', 'h@GO:0070822', 'h@GO:0030869', 'h@GO:0071339', 'h@GO:0000446', 'h@GO:0071819', 'h@GO:0071177', 'h@GO:0043189', 'h@GO:0097504', 'h@GO:0016606', 'h@GO:0042272', 'h@GO:0016591', 'h@GO:0035363', 'h@GO:0016592', 'h@GO:0070556', 'h@GO:0000811', 'h@GO:0000418', 'h@GO:0090574', 'h@GO:0005665', 'h@GO:0043240', 'h@GO:0090572', 'h@GO:0043494', 'h@GO:0005674', 'h@GO:0071821', 'h@GO:0000119', 'h@GO:0090570', 'h@GO:0035062', 'h@GO:0044665', 'h@GO:0071920', 'h@GO:0032039', 'h@GO:0035097', 'h@GO:0071176', 'h@GO:0090568', 'h@GO:0005672', 'h@GO:0070461', 'h@GO:0070722', 'h@GO:0032783', 'h@GO:0072487', 'h@GO:0070210', 'h@GO:0005848', 'h@GO:0070311', 'h@GO:1990713', 'h@GO:1990467', 'h@GO:0071181', 'h@GO:0017054', 'h@GO:0033276', 'h@GO:0000509', 'h@GO:0000124', 'h@GO:0030914', 'h@GO:0071601', 'h@GO:0070691', 'h@GO:0070985', 'h@GO:0005656', 'h@GO:0016581', 'h@GO:0070264', 'h@GO:0070211', 'h@GO:1990468', 'h@GO:0032021', 'h@GO:0033698', 'h@GO:0005671', 'h@GO:0002928', 'h@GO:0071162', 'h@GO:0005677', 'h@GO:0005849', 'h@GO:0032777', 'h@GO:0033100', 'h@GO:0016607', 'h@GO:0000125', 'h@GO:0008023', 'h@GO:0036409', 'h@GO:0005654', 'h@GO:0005675', 'h@GO:1990711', 'h@GO:0044451', 'h@GO:0035101', 'h@GO:0005669', 'h@GO:0000118', 'h@GO:0035098', 'h@GO:0010445', 'h@GO:0031261', 'h@GO:1990251'}, {'p@GO:0017076', 'p@GO:0000166', 'p@GO:0032563', 'p@GO:1902249', 'p@GO:0010181', 'p@GO:0032556', 'p@GO:0097216', 'p@GO:0050656', 'p@GO:0030553', 'p@GO:0032559', 'p@GO:0016208', 'p@GO:0051288', 'p@GO:0070403', 'p@GO:0000774', 'p@GO:0005524', 'p@GO:0035438', 'p@GO:0019003', 'p@GO:0032554', 'p@GO:0070402', 'p@GO:0032562', 'p@GO:0032553', 'p@GO:0032557', 'p@GO:0032555', 'p@GO:0019002', 'p@GO:0070401', 'p@GO:1901640', 'p@GO:0002134', 'p@GO:0070404', 'p@GO:0032566', 'p@GO:0050660', 'p@GO:1901641', 'p@GO:0019103', 'p@GO:0051287', 'p@GO:0005525', 'p@GO:0098546', 'p@GO:0002135', 'p@GO:0032560', 'p@GO:0032567', 'p@GO:0032564', 'p@GO:0008327', 'p@GO:0019001', 'p@GO:0030554', 'p@GO:0030551', 'p@GO:0043531', 'p@GO:0030552', 'p@GO:0071949', 'p@GO:0032558', 'p@GO:0071950', 'p@GO:0032565', 'p@GO:0061507', 'p@GO:0050661', 'p@GO:0032561', 'p@GO:0010429', 'p@GO:0032552', 'p@GO:0010428'})
%time np.sum(ppi_df.apply(lambda x: not x['annotations_A'].isdisjoint(pair_prop[0]) and x['annotations_B'].isdisjoint(pair_prop[1]), axis=1))
%time np.sum(ppi_df.annotations_A.map(lambda x: not x.isdisjoint(pair_prop[0])) & ppi_df.annotations_B.map(lambda x: x.isdisjoint(pair_prop[1])))

CPU times: user 128 ms, sys: 0 ns, total: 128 ms
Wall time: 127 ms
CPU times: user 9.78 ms, sys: 0 ns, total: 9.78 ms
Wall time: 9.17 ms


2368

In [111]:
def count_presences_propagated(pair_prop, interaction_dataframe, column_A,
                               column_B, merged_annotations):
    """
    columns must be host, pathogen order
    """
    """Comparison of methods

    ALL PAIRS
    %time np.sum([np.sum(interaction_dataframe['annotation_pairs'].apply(lambda x: i in x)) for i in list(itertools.product(pair_prop[0], pair_prop[1]))])
    %time np.sum(interaction_dataframe.apply(lambda x: not x['annotations_A'].isdisjoint(pair_prop[0]) and not x['annotations_B'].isdisjoint(pair_prop[1]), axis=1))
    %time np.sum(merged_annotations.map(lambda x: not x.isdisjoint(pair_prop[0]) and not x.isdisjoint(pair_prop[1])))

    CPU times: user 138 ms, sys: 30 µs, total: 138 ms
    Wall time: 136 ms
    CPU times: user 105 ms, sys: 0 ns, total: 105 ms
    Wall time: 105 ms
    CPU times: user 8.65 ms, sys: 0 ns, total: 8.65 ms
    Wall time: 8.46 ms

    ONLY 1 LABEL

    """

    # count all ppis where any pair occurs
    pair_count = np.sum(
        merged_annotations.map(
            lambda x: not x.isdisjoint(pair_prop[0]) and not x.isdisjoint(pair_prop[1])
        ))

    # count ppis where only 1 label occurs: P(X | Y') or N10
    label_one_count_exclusive = np.sum(interaction_dataframe[column_A].map(
        lambda x: not x.isdisjoint(pair_prop[0])
    ) & interaction_dataframe[column_B].map(
        lambda x: x.isdisjoint(pair_prop[1])))
    label_two_count_exclusive = np.sum(interaction_dataframe[column_A].map(
        lambda x: x.isdisjoint(pair_prop[0])
    ) & interaction_dataframe[column_B].map(
        lambda x: not x.isdisjoint(pair_prop[1])))

    # count ppis where 1 label occurs, regardless of other label in pair:
    # P(X) or N1+
    label_one_count = np.sum(
        merged_annotations.apply(lambda x: not x.isdisjoint(pair_prop[0])))
    label_two_count = np.sum(
        merged_annotations.apply(lambda x: not x.isdisjoint(pair_prop[1])))
    # option to not discern between location of label (i.e. host or pathogen)
    # in case there would be identical labels in host and pathogen proteins
    # label_one_count = np.sum(interaction_dataframe[column_A].map(
    #     lambda x: not x.isdisjoint(pair_prop[0])))
    # label_two_count = np.sum(interaction_dataframe[column_B].map(
    #     lambda x: not x.isdisjoint(pair_prop[1])))

    # count ppis lacking either term: P(X',Y') or N00
    absent_count = np.sum(
        merged_annotations.map(lambda x: x.isdisjoint(pair_prop[0])) &
        merged_annotations.map(lambda x: x.isdisjoint(pair_prop[1])))
    # option to not discern between location of label (i.e. host or pathogen)
    # absent_count = np.sum(interaction_dataframe[column_A].map(
    #     lambda x: x.isdisjoint(pair_prop[0])) & interaction_dataframe[column_B]
    #                       .map(lambda x: x.isdisjoint(pair_prop[1])))

    # total count of terms
    total_count = interaction_dataframe.shape[0]

    return pair_count, label_one_count_exclusive, label_two_count_exclusive, label_one_count, label_two_count, absent_count, total_count

In [113]:
pair_prop = ({'h@GO:0042382', 'h@GO:0071182', 'h@GO:1990483', 'h@GO:0032221', 'h@GO:0016605', 'h@GO:0061793', 'h@GO:0044545', 'h@GO:0008024', 'h@GO:0015030', 'h@GO:0034967', 'h@GO:0035267', 'h@GO:0090571', 'h@GO:0000438', 'h@GO:0048188', 'h@GO:0070176', 'h@GO:0046695', 'h@GO:0005847', 'h@GO:0000123', 'h@GO:0071178', 'h@GO:0070775', 'h@GO:1990331', 'h@GO:0070449', 'h@GO:0061773', 'h@GO:0071180', 'h@GO:0005673', 'h@GO:0070824', 'h@GO:0016593', 'h@GO:0016604', 'h@GO:1902562', 'h@GO:0070692', 'h@GO:0005666', 'h@GO:0036410', 'h@GO:0035061', 'h@GO:1990512', 'h@GO:0070823', 'h@GO:0034740', 'h@GO:0070776', 'h@GO:0016580', 'h@GO:0000420', 'h@GO:0000508', 'h@GO:0044666', 'h@GO:0070440', 'h@GO:0033255', 'h@GO:0070443', 'h@GO:0090573', 'h@GO:0032044', 'h@GO:0072589', 'h@GO:0070847', 'h@GO:0000443', 'h@GO:0071179', 'h@GO:1990826', 'h@GO:0070439', 'h@GO:0071175', 'h@GO:0070822', 'h@GO:0030869', 'h@GO:0071339', 'h@GO:0000446', 'h@GO:0071819', 'h@GO:0071177', 'h@GO:0043189', 'h@GO:0097504', 'h@GO:0016606', 'h@GO:0042272', 'h@GO:0016591', 'h@GO:0035363', 'h@GO:0016592', 'h@GO:0070556', 'h@GO:0000811', 'h@GO:0000418', 'h@GO:0090574', 'h@GO:0005665', 'h@GO:0043240', 'h@GO:0090572', 'h@GO:0043494', 'h@GO:0005674', 'h@GO:0071821', 'h@GO:0000119', 'h@GO:0090570', 'h@GO:0035062', 'h@GO:0044665', 'h@GO:0071920', 'h@GO:0032039', 'h@GO:0035097', 'h@GO:0071176', 'h@GO:0090568', 'h@GO:0005672', 'h@GO:0070461', 'h@GO:0070722', 'h@GO:0032783', 'h@GO:0072487', 'h@GO:0070210', 'h@GO:0005848', 'h@GO:0070311', 'h@GO:1990713', 'h@GO:1990467', 'h@GO:0071181', 'h@GO:0017054', 'h@GO:0033276', 'h@GO:0000509', 'h@GO:0000124', 'h@GO:0030914', 'h@GO:0071601', 'h@GO:0070691', 'h@GO:0070985', 'h@GO:0005656', 'h@GO:0016581', 'h@GO:0070264', 'h@GO:0070211', 'h@GO:1990468', 'h@GO:0032021', 'h@GO:0033698', 'h@GO:0005671', 'h@GO:0002928', 'h@GO:0071162', 'h@GO:0005677', 'h@GO:0005849', 'h@GO:0032777', 'h@GO:0033100', 'h@GO:0016607', 'h@GO:0000125', 'h@GO:0008023', 'h@GO:0036409', 'h@GO:0005654', 'h@GO:0005675', 'h@GO:1990711', 'h@GO:0044451', 'h@GO:0035101', 'h@GO:0005669', 'h@GO:0000118', 'h@GO:0035098', 'h@GO:0010445', 'h@GO:0031261', 'h@GO:1990251'}, {'p@GO:0017076', 'p@GO:0000166', 'p@GO:0032563', 'p@GO:1902249', 'p@GO:0010181', 'p@GO:0032556', 'p@GO:0097216', 'p@GO:0050656', 'p@GO:0030553', 'p@GO:0032559', 'p@GO:0016208', 'p@GO:0051288', 'p@GO:0070403', 'p@GO:0000774', 'p@GO:0005524', 'p@GO:0035438', 'p@GO:0019003', 'p@GO:0032554', 'p@GO:0070402', 'p@GO:0032562', 'p@GO:0032553', 'p@GO:0032557', 'p@GO:0032555', 'p@GO:0019002', 'p@GO:0070401', 'p@GO:1901640', 'p@GO:0002134', 'p@GO:0070404', 'p@GO:0032566', 'p@GO:0050660', 'p@GO:1901641', 'p@GO:0019103', 'p@GO:0051287', 'p@GO:0005525', 'p@GO:0098546', 'p@GO:0002135', 'p@GO:0032560', 'p@GO:0032567', 'p@GO:0032564', 'p@GO:0008327', 'p@GO:0019001', 'p@GO:0030554', 'p@GO:0030551', 'p@GO:0043531', 'p@GO:0030552', 'p@GO:0071949', 'p@GO:0032558', 'p@GO:0071950', 'p@GO:0032565', 'p@GO:0061507', 'p@GO:0050661', 'p@GO:0032561', 'p@GO:0010429', 'p@GO:0032552', 'p@GO:0010428'})

count_presences_propagated(pair_prop, interaction_dataframe, 'annotations_A', 'annotations_B', merged_annotations)

(91, 2368, 212, 2459, 303, 5080, 7751)

In [4]:
# Create Gene Ontology dictionaries
print('Creating Gene Ontology dictionaries...')
obo_path = Path('/media/pieter/DATA/Wetenschap/Doctoraat/projects/host-pathogen-ppi-analysis/data/raw/go_data/go.obo')
go_dict = obo_tools.importOBO(obo_path, ignore_part_of=False)

# TODO: allow selection of GO namespaces
obo_tools.buildGOtree(
    go_dict, root_nodes=['GO:0008150', 'GO:0005575', 'GO:0003674'])

Creating Gene Ontology dictionaries...
Retrieved 47179 GO terms from /media/pieter/DATA/Wetenschap/Doctoraat/projects/host-pathogen-ppi-analysis/data/raw/go_data/go.obo 

Adding secondary GO identifiers...



In [18]:
pair = ('h@IPR008936', 'p@GO:0016020')
new_pair = []
for term in pair:
    if 'GO' in term:
        prefix = term[:2]
        go_object = go_dict.get(term[2:])
        if None:
            new_pair.append(term)
        else:
            child_terms = [prefix + i for i in go_object.recursive_children]
            propagation_set = set(child_terms) | set([term])
            new_pair.append(propagation_set)
    else:
        new_pair.append({term})
prop_pair = tuple(new_pair)

In [19]:
merged_annotations_A = pairwise.merge_annotations(
    ppi_df, ['GO_xref_A', 'interpro_xref_A'])
labeled_merged_annotations_A = pairwise.add_hp_label(merged_annotations_A, 'h')
ppi_df['annotations_A'] = labeled_merged_annotations_A

merged_annotations_B = pairwise.merge_annotations(
    ppi_df, ['GO_xref_B', 'interpro_xref_B'])
labeled_merged_annotations_B = pairwise.add_hp_label(merged_annotations_B, 'p')
ppi_df['annotations_B'] = labeled_merged_annotations_B

In [20]:
column_A = 'annotations_A'
column_B = 'annotations_B'
columns = [column_A, column_B]
    # replace NaNs with empty sets
for i in columns:
    interaction_dataframe.loc[interaction_dataframe[
        i].isnull(), i] = interaction_dataframe.loc[interaction_dataframe[
            i].isnull(), i].apply(lambda x: set())
interaction_dataframe['annotation_pairs'] = interaction_dataframe[columns].apply(
        lambda x: set([i for i in itertools.product(x[column_A], x[column_B])]),
        axis=1)
merged_annotations = pairwise.merge_annotations(interaction_dataframe, columns)

In [21]:
pairs_dict = pairwise.propagate_pairs([pair], go_dict)

In [22]:
pairs_dict

{('h@IPR008936', 'p@GO:0016020'): ({'h@IPR008936'},
  {'p@GO:0035453',
   'p@GO:1990702',
   'p@GO:0034449',
   'p@GO:0016013',
   'p@GO:0046691',
   'p@GO:0030089',
   'p@GO:0030123',
   'p@GO:0030085',
   'p@GO:0036513',
   'p@GO:0033111',
   'p@GO:0098892',
   'p@GO:0014701',
   'p@GO:1990694',
   'p@GO:0030937',
   'p@GO:0034448',
   'p@GO:0031095',
   'p@GO:0015008',
   'p@GO:0036421',
   'p@GO:0032585',
   'p@GO:0031676',
   'p@GO:0099030',
   'p@GO:0055035',
   'p@GO:0071130',
   'p@GO:0033116',
   'p@GO:0097585',
   'p@GO:1990332',
   'p@GO:0045261',
   'p@GO:0070020',
   'p@GO:0098929',
   'p@GO:0044459',
   'p@GO:0008136',
   'p@GO:0106084',
   'p@GO:0031166',
   'p@GO:0031311',
   'p@GO:0070719',
   'p@GO:0042613',
   'p@GO:0031352',
   'p@GO:0005745',
   'p@GO:0000506',
   'p@GO:0017090',
   'p@GO:1990695',
   'p@GO:0071117',
   'p@GO:0098889',
   'p@GO:0070110',
   'p@GO:1990913',
   'p@GO:0000139',
   'p@GO:0034272',
   'p@GO:0045284',
   'p@GO:1990154',
   'p@GO:0035355'

In [28]:
merged_annotations = merged_annotations[:1]
interaction_dataframe = interaction_dataframe[:1]

In [29]:
for pair, propagated_pair in pairs_dict.items():

    pair_count, label_one_count_exclusive, label_two_count_exclusive, label_one_count, label_two_count, absent_count, total_count = pairwise.count_presences_propagated(
                    propagated_pair, interaction_dataframe, column_A, column_B,
                    merged_annotations)
    print(pairwise.count_presences_propagated(
                    propagated_pair, interaction_dataframe, column_A, column_B,
                    merged_annotations))

(1, 0, 0, 1, 1, 0, 1)


In [30]:
print(pair_count, label_one_count_exclusive,label_two_count_exclusive, absent_count)

1 0 0 0


In [31]:
def _calc_chi2(n11, n10, n01, n00):
    print(n11, n10, n01, n00)
    print(type(n11))
    contingency_table = np.array([[n11, n01], [n10, n00]])

    if 0 in contingency_table:
        print(contingency_table)

    chi2, p, df, ex = scipy.stats.chi2_contingency(
        contingency_table, correction=False, lambda_="pearson")
    d = {'chi2': chi2, 'p-value': p, 'df': df, 'exp': ex}
    return d

In [32]:
f = _calc_chi2(pair_count, label_one_count_exclusive,label_two_count_exclusive, absent_count)

1 0 0 0
<class 'numpy.int64'>
[[1 0]
 [0 0]]


ValueError: The internally computed table of expected frequencies has a zero element at (0, 1).

In [34]:
print(len(pairs))

447228


In [54]:
d = {'chi2' : {'pair1' : {'chi2_p':1, 'chi2_df': 2}, 'pair2': {'chi2_p':3, 'chi2_df': 4}}, 'phi': {'pair1' : {'chi2_p':11, 'chi2_df': 22}, 'pair2': {'chi2_p':33, 'chi2_df': 44}}}

In [55]:
pd.concat([pd.DataFrame.from_dict(d, orient='index').add_prefix(name + '_') for name, d in d.items()], axis=1)

,chi2_chi2_p,chi2_chi2_df,phi_chi2_p,phi_chi2_df
pair1,1,2,11,22
pair2,3,4,33,44


In [56]:
pd.DataFrame.from_dict({'pair1' : [1,2,3], 'pair2': [4,5,6]}, orient='index')

,0,1,2
pair1,1,2,3
pair2,4,5,6


In [57]:
d2 = {'pair1': {'chi2': {'chi2_p':1, 'chi2_df': 2}, 'phi': {'chi2_p':11, 'chi2_df': 22}}, 'pair2': {'chi2': {'chi2_p':3, 'chi2_df': 4}, 'phi': {'chi2_p':33, 'chi2_df': 44}}}

In [60]:
[pd.DataFrame.from_dict(d, orient='index').add_prefix(name + '_') for name, d in d2.items()]

[      pair1_chi2_p  pair1_chi2_df
 chi2             1              2
 phi             11             22,       pair2_chi2_p  pair2_chi2_df
 chi2             3              4
 phi             33             44]

In [62]:
pd.DataFrame.from_dict(d, orient='columns')

,chi2,phi
pair1,"{'chi2_p': 1, 'chi2_df': 2}","{'chi2_p': 11, 'chi2_df': 22}"
pair2,"{'chi2_p': 3, 'chi2_df': 4}","{'chi2_p': 33, 'chi2_df': 44}"


In [63]:
pair

('h@IPR008936', 'p@GO:0016020')

In [66]:
go_dict['p@GO:0016020'[2:]].depth

1

In [73]:
{'protein_A': go_dict[pair[0][2:]].depth if 'GO' in pair[0] else None, 'protein_B': go_dict[pair[1][2:]].depth if 'GO' in pair[1] else None}

{'protein_A': None, 'protein_B': 1}

In [72]:
go_dict['GO:0016020'].depth

1